# Advent of Code 2015
## In the style of Peter Norvig

First some imports and utility Functions.
Most of Peter's utility functions have moved into <a href=file:///Volumes/generic/apuri/g/misc/python/2015-advent-of-code/util.py> util.py </a>

In [138]:
import time
import itertools
import math
import collections
import re
import operator
import numpy
import sys
import string
import re
import functools
import util as u
import more_itertools as mi
#from fastcore.test import test_eq  # fastcore is library I want to learn

def test_eq(actual,expected):  # glorified assert, but when it fails you get some useful info
    if actual != expected:
        print('actual=',actual)
        print('expected=', expected)
        assert False

def Input(n, dir = 'data'):
        return open(f'{dir}/{n:02d}.txt').read()

def remove_all(elt, xs):
        return [x for x in xs if x != elt]



## <a href="https://adventofcode.com/2015/day/1">Day 1 </a> :  Not quite Lisp

In [114]:
def bracketmap(bracketstr):
        return [1 if c == '(' else -1 for c in bracketstr ]

def level(bracketstr):
        return sum(bracketmap(bracketstr))
    
# tests
test_eq([level(s) for s in '(())  ()()  (((  (()(()(  ))(((((  ())  ))(  )))  )())())'.split()],
       [0, 0, 3, 3, 3, -1, -1, -3, -3])

# puzzle
test_eq(level(mi.first(u.Input(1))), 138)

In [45]:
# Part 2

def steps_to_basement(bracketstr):
        "2020.09.16 1905-2005 - mostly to learn doctest"
        return 2+mi.last(enumerate(itertools.takewhile(lambda x: x >= 0, 
                                                       itertools.accumulate(bracketmap(bracketstr)))),
                         (-1,0))[0] # the [0] picks out the enumeration number 

# tests
test_eq([steps_to_basement(s) for s in '()())  )'.split()], [5, 1])

# puzzle
test_eq(steps_to_basement(Input(1, 'data').strip()), 1771)


## <a href="https://adventofcode.com/2015/day/2"> Day 2 </a>: I Was Told There Would Be No Math

In [92]:
def box_area_to_wrap(dims):
        """
        dims is a tuple of box dimensions (length l, width w, and height h)
        to wrap we need the surface area + slack = area of smallest side
        """
        sides = [x*y for x, y in itertools.combinations(dims, 2)]
        return sum(sides) * 2 + min(sides)

# tests
test_eq([box_area_to_wrap(dims) for dims in [(2,3,4), (1,1,10)]], 
        [58, 43])
 
# puzzle
test_eq(sum(map(box_area_to_wrap, u.Input(2, u.integers))), 1586300)


In [100]:
# Part 2
def ribbon(dims):
        """
        2020.09.16 2005-2045

        A present with dimensions 2x3x4 needs 
        2+2+3+3 = 10 feet of ribbon to wrap the present plus 
        2*3*4 = 24 feet of ribbon for the bow, for a total of 34 feet.
        
        A present with dimensions 1x1x10 needs 
        1+1+1+1 = 4 feet of ribbon to wrap the present plus 
        1*1*10 = 10 feet of ribbon for the bow, for a total of 14 feet.
       """
        dims = sorted(dims)
        return sum(dims[:2]) * 2 + math.prod(dims)
    
# tests
test_eq(list(map(ribbon, [(2,3,4),(1,1,10)])),  [34, 14])

# puzzle
test_eq(sum(ribbon(dims) for dims in u.Input(2, u.integers)), 3737498)


   

## <a href="https://adventofcode.com/2015/day/3"> Day 3 </a>: Perfectly Spherical Houses in a Vacuum

In [130]:
P = complex

def pathremap(path):
        "convert the arrows into 'steps' in the complex plane"
        d = {   '^' : P(0,1), '>' : P(1),
                'v' : P(0,-1), '<': P(-1) }
        return map(d.get, path)

def distinct_points(path):
        "2020.09.16 2045-2105 = 20m"
        return len(set(itertools.accumulate(itertools.chain([P(0)], pathremap(path)))))


test_eq([distinct_points(s) for s in ['>', '^>v<', '^v^v^v^v^v']],
        [2, 4, 2])
test_eq(distinct_points(mi.first(u.Input(3))), 2565)


In [144]:
def two_santas(path):
        "2020.09.15 2105-2135 = 30m"
        paths = mi.distribute(2, pathremap(path))  # one path for each santa
        paths = [itertools.accumulate(itertools.chain([P(0)],path)) for path in paths]
        return len(set(itertools.chain(*paths)))

test_eq([two_santas(s) for s in ['^v', '^>v<', '^v^v^v^v^v']],
    [3, 3, 11])
test_eq(two_santas(mi.first(u.Input(3))), 2639)


## <a href="https://adventofcode.com/2015/day/4"> Day 4 </a>: The Ideal Stocking Stuffer

These can take a /while/ to evaluate 

In [178]:
import hashlib

def functional_find_num(key, nzeros=5):
        # NOT USED : just here to show that sometimes fp seems less clear
        return mi.first(i
            for i, m in enumerate(map(u.compose(hashlib.md5,
                                            str.encode, 
                                            functools.partial(operator.add, key), 
                                                    # prepend key
                                            str),   # convert count to a str
                                  itertools.count()))
                if m.hexdigest()[:nzeros] == '0'*nzeros)

                
def find_num(key, nzeros=5):
        "2020.09.16 2135-2200 = 25m"
        for i in itertools.count():
                s = key+str(i)
                m = hashlib.md5(s.encode())
                if m.hexdigest()[:nzeros] == '0'*nzeros: return i

       
# tests
test_eq(u.mapt(find_num, ['abcdef', 'pqrstuv']),
        (609043, 1048970))

# puzzle
%time test_eq(find_num('iwrupvqb'), 346386)

CPU times: user 605 ms, sys: 1.64 ms, total: 607 ms
Wall time: 609 ms


In [177]:
# takes about 20s on my machine
%time test_eq(find_num('iwrupvqb', 6), 9958218)

CPU times: user 21.4 s, sys: 21 ms, total: 21.4 s
Wall time: 21.5 s
